In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import scipy
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

In [2]:
customer_segment = pd.read_csv('../data/customer_segments.csv')
customer_segment.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date,Cash /Cashless Type,...,State,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment
0,CS000018,CS000018,Unknown,NaN,Unknown,Unknown,Unknown,NaN,NaN,NaN,...,tamil nadu,CS000018,186,2,1,0,49.75,0,2,1
1,E10007,E10007,Retail,Camp-Outdoor,Unknown,Unknown,Unknown,NaN,NaN,NaN,...,telangana,E10007,1215,0,1,0,1083.77,0,0,0
2,E10013,E10013,Retail,Camp-Outdoor,Unknown,Unknown,Unknown,NaN,NaN,NaN,...,telangana,E10013,1052,0,2,0,1001.29,0,0,0
3,E10013,E10013,Retail,Camp-Outdoor,Unknown,Unknown,Unknown,NaN,NaN,NaN,...,telangana,E10013,1052,0,2,0,1001.29,0,0,0
4,E10014,E10014,Retail,Camp-Outdoor,Mr.,Unknown,Unknown,NaN,NaN,NaN,...,telangana,E10014,1290,0,1,0,785.21,0,0,0


In [5]:
customer_segment.shape

(492314, 59)

In [6]:
customer_segment.columns

Index(['Business Partner', 'Customer No.', 'Partner Type', 'Data Origin',
       'Title', 'Marital Status', 'Occupation', 'Date of Birth', 'Death date',
       'Cash /Cashless Type', 'Claim No.', 'Cust Type', 'Expiry Date',
       'Gate Pass Date', 'Gate Pass Time', 'Insurance Company', 'Invoice Date',
       'Invoice No', 'Invoice Time', 'Job Card No', 'JobCard Date',
       'JobCard Time', 'KMs Reading', 'Labour Total', 'Make', 'Misc Total',
       'Model', 'ODN No.', 'OSL Total', 'Order Type', 'Parts Total',
       'Pin code', 'Plant', 'Plant Name1', 'Policy no.', 'Print Status',
       'Recovrbl Exp', 'Regn No', 'Service Advisor Name', 'Technician Name',
       'Total Amt Wtd Tax.', 'Total Value', 'User ID', 'Invoice Day',
       'Invoice Month', 'Invoice Year', 'Day of the week', 'Division Name',
       'District', 'State', 'CustomerID', 'Recency', 'RecencyCluster',
       'Frequency', 'FrequencyCluster', 'Revenue', 'RevenueCluster',
       'OverallScore', 'Segment'],
      dtype=

In [34]:
customer_segment['Regn No'].isnull().sum()

16

In [35]:
customer_segment = customer_segment[customer_segment['Regn No'].notna()]
customer_segment['Regn No'].isnull().sum()

0

In [50]:
customer_segment.shape

(492298, 59)

In [51]:
customer_regn = customer_segment.groupby('Customer No.')['Regn No'].nunique().reset_index()

customer_regn.columns = ['Customer No.', 'Cars owned']

customer_segment = customer_segment.merge(customer_regn, on='Customer No.')

customer_segment.shape

(492298, 60)

In [54]:
customer_segment.columns

Index(['Business Partner', 'Customer No.', 'Partner Type', 'Data Origin',
       'Title', 'Marital Status', 'Occupation', 'Date of Birth', 'Death date',
       'Cash /Cashless Type', 'Claim No.', 'Cust Type', 'Expiry Date',
       'Gate Pass Date', 'Gate Pass Time', 'Insurance Company', 'Invoice Date',
       'Invoice No', 'Invoice Time', 'Job Card No', 'JobCard Date',
       'JobCard Time', 'KMs Reading', 'Labour Total', 'Make', 'Misc Total',
       'Model', 'ODN No.', 'OSL Total', 'Order Type', 'Parts Total',
       'Pin code', 'Plant', 'Plant Name1', 'Policy no.', 'Print Status',
       'Recovrbl Exp', 'Regn No', 'Service Advisor Name', 'Technician Name',
       'Total Amt Wtd Tax.', 'Total Value', 'User ID', 'Invoice Day',
       'Invoice Month', 'Invoice Year', 'Day of the week', 'Division Name',
       'District', 'State', 'CustomerID', 'Recency', 'RecencyCluster',
       'Frequency', 'FrequencyCluster', 'Revenue', 'RevenueCluster',
       'OverallScore', 'Segment', 'Cars owned']

In [3]:
customer_segment['has_policy'] = customer_segment['Insurance Company'].notnull().astype(int)

customer_segment['has_policy'].value_counts()

0    468723
1     23591
Name: has_policy, dtype: int64

In [4]:
relevant_features = ['Data Origin', 'Cust Type', 'KMs Reading', 'Make',
       'Model','Order Type', 'Plant Name1', 'Total Amt Wtd Tax.', 
                     'District', 'State', 'has_policy', 'Segment']

In [5]:
customer_segment[customer_segment['Segment']==2]['has_policy'].value_counts()

0    5634
1      41
Name: has_policy, dtype: int64

#### Not grouping by customer ID because for every new customer, we are trying to predict the segment of that customer to be low, mid or high. As the customer is coming for the first time, the frequency and the amount they will generate will be low. Hence, for new customers, the model will always predict them to be of the lower segment. 

#### We are keeping the different services by the same customers as they are in the data set and not grouping them, so that the model considers them to be different customers entirely.

In [45]:
customer_model_data = customer_segment[relevant_features]
customer_model_data.head()

,Data Origin,Cust Type,KMs Reading,Make,Model,Order Type,Plant Name1,Total Amt Wtd Tax.,District,State,has_policy,Segment
0,NaN,Retail,3000,HONDA,MOBILIO,Mechanical,NERKUNDRAM,49.75,kanchipuram,tamil nadu,1,1
1,Camp-Outdoor,Retail,172979,MAHINDRA & MAHINDRA,BOLERO PIK UP,Running Repairs,BALANAGAR,1083.77,hyderabad,telangana,0,0
2,Camp-Outdoor,Retail,42413,MARUTI SUZUKI,WAGONR,SMC Value Package,BALANAGAR,1001.29,hyderabad,telangana,0,0
3,Camp-Outdoor,Retail,42413,MARUTI SUZUKI,WAGONR,SMC Redemption,BALANAGAR,0.00,hyderabad,telangana,0,0
4,Camp-Outdoor,Retail,79041,MARUTI SUZUKI,WAGONR,Paid Service,MADEENAGUDA,785.21,hyderabad,telangana,0,0


In [46]:
customer_model_data['Segment'].isnull().sum()

0

In [47]:
customer_model_data['Model'].isnull().sum()

1525

In [48]:
#fill null values
customer_model_data['Data Origin'].fillna('Data Origin Unknown', inplace=True)
customer_model_data['Model'].fillna('Model Unknown', inplace=True)

In [27]:
customer_model_data.columns

Index(['Data Origin', 'Cust Type', 'KMs Reading', 'Make', 'Model',
       'Order Type', 'Plant Name1', 'Total Amt Wtd Tax.', 'District', 'State',
       'has_policy', 'Segment'],
      dtype='object')

In [2]:
customer_model_data = pd.read_csv('../data/customer_model_data.csv')

In [ ]:
data_origin_enc = pd.get_dummies(customer_model_data['Data Origin'],prefix = 'data_origin_')

cust_type_enc = pd.get_dummies(customer_model_data['Cust Type'],prefix = 'cust_type_')

make_enc = pd.get_dummies(customer_model_data['Make'],prefix = 'make_')

model_enc =  pd.get_dummies(customer_model_data['Model'],prefix = 'model_')

ot_enc =  pd.get_dummies(customer_model_data['Order Type'],prefix = 'ot_')

plant_enc =  pd.get_dummies(customer_model_data['Plant Name1'],prefix = 'plant_')

state_enc =  pd.get_dummies(customer_model_data['State'],prefix = 'state_')

district_enc =  pd.get_dummies(customer_model_data['District'],prefix = 'district_').astype(np.int8)

data_encoded = pd.concat([data_origin_enc, cust_type_enc, make_enc, model_enc, ot_enc, plant_enc, state_enc, district_enc],
                         axis=1)

data_encoded = pd.concat([data_encoded, customer_model_data['KMs Reading'], customer_model_data['Total Amt Wtd Tax.'],
                         customer_model_data['has_policy'], customer_model_data['Segment']], axis=1)

In [ ]:
data_encoded.shape

In [23]:
input_data = ['Camp-Outdoor', 'Retail', 3000, 'HONDA', 'MOBILIO', 'Mechanical', 'BALANAGAR', 500, 'hyderabad', 'telangana', 
              '0']
input_df = pd.DataFrame({
    'Data Origin': input_data[0],
    'Cust Type': input_data[1],
    'KMs Reading': input_data[2], 
    'Make': input_data[3], 
    'Model': input_data[4],
    'Order Type': input_data[5], 
    'Plant Name1': input_data[6], 
    'Total Amt Wtd Tax.': input_data[7], 
    'District': input_data[8], 
    'State': input_data[9],
    'has_policy': input_data[10]
}, index=[0])
categorical_cols = ['Data Origin', 'Cust Type', 'Make', 'Model', 'Order Type', 'Plant Name1', 'District', 'State']
input_data_encoded = pd.get_dummies(input_df[categorical_cols])
input_data_encoded[['KMs Reading', 'Total Amt Wtd Tax.', 'has_policy']] = input_df[['KMs Reading', 'Total Amt Wtd Tax.', 'has_policy']]
# Get missing columns in the training test
missing_cols = set( data_encoded.columns ) - set( input_data_encoded.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    input_data_encoded[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
input_data_encoded = input_data_encoded[data_encoded.columns]

In [24]:
input_data_encoded

,data_origin__Camp-Outdoor,data_origin__Data Origin Unknown,cust_type__Corporate others,cust_type__Corporate- M&M,cust_type__Fleets,cust_type__Franchise,cust_type__Insurance,cust_type__MFCWL,cust_type__Retail,make__AUDI,...,district__zirakhpur,district__zirakpur,district__zirkapur,district__zirkpur,district__zirkpur mohali,district__zirkpur punjab,KMs Reading,Total Amt Wtd Tax.,has_policy,Segment
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3000,500,0,0


In [8]:
customer_model_data.columns

Index(['Data Origin', 'Cust Type', 'KMs Reading', 'Make', 'Model',
       'Order Type', 'Plant Name1', 'Total Amt Wtd Tax.', 'District', 'State',
       'has_policy', 'Segment'],
      dtype='object')

In [11]:
#normalize numerical features
numerica_df = data_encoded[['KMs Reading', 'Total Amt Wtd Tax.']]

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(numerica_df)
df_normalized = pd.DataFrame(np_scaled, columns = ['KMs Reading', 'Total Amt Wtd Tax.'])
data_encoded[['KMs Reading', 'Total Amt Wtd Tax.']] = df_normalized[['KMs Reading', 'Total Amt Wtd Tax.']]
data_encoded[['KMs Reading', 'Total Amt Wtd Tax.']].head() 

,KMs Reading,Total Amt Wtd Tax.
0,0.000300,0.000288
1,0.017298,0.001883
2,0.004241,0.001756
3,0.004241,0.000211
4,0.007904,0.001422


In [12]:
X = data_encoded.loc[:, data_encoded.columns != 'Segment']
y = data_encoded['Segment']

In [13]:
X.shape

(492314, 4145)

In [50]:
data_iterator = pd.read_csv('../data/data_encoded.csv',chunksize=10000)

classifier = SGDClassifier(n_jobs=-1,verbose=1)

In [ ]:
i=0
for data_set in data_iterator:
    i+=1
    X = data_set.iloc[:,:-1]
    y = data_set['Segment']
    print('Iteration: '+str(i))
    classifier.partial_fit(X,y,classes=[0,1,2])

Iteration: 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


-- Epoch 1-- Epoch 1-- Epoch 1


Norm: 661682.68, NNZs: 198, Bias: -1217.356008, T: 10000, Avg. loss: 7043186477.904287
Total training time: 0.25 seconds.
Norm: 83245.96, NNZs: 240, Bias: 1278.481296, T: 10000, Avg. loss: 11785207931.481983Norm: 120972.08, NNZs: 239, Bias: 171.581329, T: 10000, Avg. loss: 13377103691.427748
Total training time: 0.27 seconds.

Total training time: 0.33 seconds.


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.2s finished


Iteration: 2


...

-- Epoch 1-- Epoch 1

-- Epoch 1
Norm: 20467.71, NNZs: 330, Bias: -2143.552666, T: 10000, Avg. loss: 0.000000
Total training time: 0.12 seconds.
Norm: 1292.87, NNZs: 1634, Bias: -345.599425, T: 10000, Avg. loss: 33038909.235149
Total training time: 0.16 seconds.
Norm: 2888.12, NNZs: 1624, Bias: 2830.578972, T: 10000, Avg. loss: 41061403.462375
Total training time: 0.16 seconds.
Iteration: 31


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


-- Epoch 1-- Epoch 1-- Epoch 1


Norm: 19809.58, NNZs: 330, Bias: -2143.552666, T: 10000, Avg. loss: 0.000000
Total training time: 0.11 seconds.
Norm: 15485.35, NNZs: 1644, Bias: -353.751131, T: 10000, Avg. loss: 126234801.994490Norm: 9335.26, NNZs: 1626, Bias: 2836.388109, T: 10000, Avg. loss: 34378413.703510
Total training time: 0.14 seconds.

Total training time: 0.14 seconds.
Iteration: 32


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 19192.46, NNZs: 330, Bias: -2143.552666, T: 10000, Avg. loss: 0.000000
Total training time: 0.11 seconds.
Norm: 1956.47, NNZs: 1635, Bias: 2835.882601, T: 10000, Avg. loss: 17259868.667158
Total training time: 0.11 seconds.
Norm: 1232.49, NNZs: 1650, Bias: -354.368733, T: 10000, Avg. loss: 13320502.096312
Total training time: 0.13 seconds.


In [26]:
classifier.predict(input_data_encoded.iloc[:,:-1])

array([1])

In [46]:
def perform_cv(model, X, y):
    f1_scores = []
    X_t, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#     model.fit(X_t, y_train)
    model_pred = model.predict(X_t)
    print(classification_report(y_train, model_pred))
    f1 = f1_score(y_train, model_pred,average="weighted")
    print("F1 score: "+str(f1))
    f1_scores.append(f1)
    return f1_scores

In [47]:
data_iterator = pd.read_csv('../data/data_encoded.csv',chunksize=100000)

In [49]:
i=0
for data_set in data_iterator:
    i+=1
    X = data_set.iloc[:,:-1]
    y = data_set['Segment']
    print('Iteration: '+str(i))
    perform_cv(classifier, X, y)

KeyboardInterrupt: 